# Progetto Foundations of Computer Science (gruppo 47)

You have to work on the [University dataset](https://drive.google.com/drive/folders/1Hs3nRtK_F3h8eg59B4-TD1DEua6g8Klv?usp=sharing). It contains three different university rankings:
*  The Times Higher Education World University Ranking, shortly *Times*,
*  the Academic Ranking of World Universities, shortly *Shanghai*,
*  the Center for World University Rankings, shortly *cwur*.

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com).

### Import and read files

Importazione delle librerie utilizzate

In [1]:
import pandas as pd
import numpy as np
import re
import collections as coll

Lettura e visualizzazione dei files

#### Times
- na_values='-' per impostare i valori nulli
- thousands=',' per impostare le migliaia (nella cultura anglosassone è prassi utilizzare la virgola come separatore)

In [2]:
times=pd.read_csv('timesData.csv',na_values='-',thousands=',')
times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,20152.0,8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,2243.0,6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,11074.0,9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,15596.0,7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,NaN,94.2,7929.0,8.4,27%,45 : 55,2011


#### Shanghai

In [3]:
shanghai=pd.read_csv('shanghaiData.csv')
shanghai.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005
3,4,"University of California, Berkeley",3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005
4,5,Massachusetts Institute of Technology (MIT),4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005


#### Center of World University Ranking (CWUR)

In [4]:
cwur=pd.read_csv('cwurData.csv')
cwur.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


### Pre-processing

##### 1. Correzione di alcuni typos, incontrati nel punto 7, del dataset *times*

In [5]:
times.replace('Unted Kingdom','United Kingdom',inplace=True)
times.replace('Unisted States of America','United States of America',inplace=True)

##### 2. Nuova configurazione per il nominativo dell'università nel dataset *cwur*, per conformarlo ai restanti dataset

In [6]:
cwur.rename(columns={'institution':'university_name'},inplace=True)
cwur.head()

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


##### 3. Normalizzazione dei nomi delle università e creazione di una chiave univoca *uni_id*

Creazione di una funzione **norm_uni** per normalizzare una stringa mantenendo esclusivamente caratteri alfanumerici (in minuscolo)

In [7]:
def norm_uni(x):
    x=re.sub('[^A-Za-z0-9]+', '',str(x)).lower()
    return x 

Teoricamente il numero univoco di *uni_id*  e *university_name*  per ciascun dataset dovrebbe essere lo stesso se non ci fossero problemi

In [8]:
times['uni_id']=times['university_name'].apply(norm_uni)
len(times['uni_id'].unique())==len(times['university_name'].unique())

True

In [9]:
shanghai['uni_id']=shanghai['university_name'].apply(norm_uni)
len(shanghai['uni_id'].unique())==len(shanghai['university_name'].unique())

False

In [10]:
cwur['uni_id']=cwur['university_name'].apply(norm_uni)
len(cwur['uni_id'].unique())==len(cwur['university_name'].unique())

True

Il dataset *shanghai* probabilmente contiene qualche typos o qualche variazione di nome per la stessa università

### 1. For each university, extract from the *times* dataset the most recent and the least recent data, obtaining two separate dataframes

In [11]:
least_recent_t=times.loc[times.groupby('uni_id')['year'].idxmin()]
least_recent_t.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,uni_id
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,NaN,17422.0,15.9,15%,48 : 52,2012,aalborguniversity
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,NaN,16099.0,24.2,17%,32 : 68,2012,aaltouniversity
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,23895.0,13.6,14%,54 : 46,2011,aarhusuniversity
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,NaN,9252.0,19.2,18%,48 : 52,2012,aberystwythuniversity
2404,601-800,Adam Mickiewicz University,Poland,20.0,25.7,11.0,15.3,28.7,NaN,40633.0,15.6,1%,71 : 29,2016,adammickiewiczuniversity


In [12]:
most_recent_t=times.loc[times.groupby('uni_id')['year'].idxmax()]
most_recent_t.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,uni_id
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,NaN,17422.0,15.9,15%,48 : 52,2016,aalborguniversity
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,NaN,16099.0,24.2,17%,32 : 68,2016,aaltouniversity
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,aarhusuniversity
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,NaN,9252.0,19.2,18%,48 : 52,2016,aberystwythuniversity
2404,601-800,Adam Mickiewicz University,Poland,20.0,25.7,11.0,15.3,28.7,NaN,40633.0,15.6,1%,71 : 29,2016,adammickiewiczuniversity


### 2. For each university, compute the improvement in `income` between the least recent and the most recent data points

In [13]:
least_most_recent=pd.merge(least_recent_t,most_recent_t,suffixes=['_lr','_mr'],on='uni_id')
least_most_recent.head()

,world_rank_lr,university_name_lr,country_lr,teaching_lr,international_lr,research_lr,citations_lr,income_lr,total_score_lr,num_students_lr,...,international_mr,research_mr,citations_mr,income_mr,total_score_mr,num_students_mr,student_staff_ratio_mr,international_students_mr,female_male_ratio_mr,year_mr
0,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,NaN,17422.0,...,71.0,28.4,73.8,43.7,NaN,17422.0,15.9,15%,48 : 52,2016
1,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,NaN,16099.0,...,65.4,32.8,62.1,61.6,NaN,16099.0,24.2,17%,32 : 68,2016
2,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,23895.0,...,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016
3,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,NaN,9252.0,...,72.2,18.9,67.2,31.3,NaN,9252.0,19.2,18%,48 : 52,2016
4,601-800,Adam Mickiewicz University,Poland,20.0,25.7,11.0,15.3,28.7,NaN,40633.0,...,25.7,11.0,15.3,28.7,NaN,40633.0,15.6,1%,71 : 29,2016


In [14]:
least_most_recent['improvement']=least_most_recent['income_mr']-least_most_recent['income_lr']
least_most_improvement=least_most_recent[['university_name_lr','year_lr','year_mr','income_lr','income_mr','improvement']]
least_most_improvement.head()

,university_name_lr,year_lr,year_mr,income_lr,income_mr,improvement
0,Aalborg University,2012,2016,36.4,43.7,7.3
1,Aalto University,2012,2016,61.9,61.6,-0.3
2,Aarhus University,2011,2016,61.5,68.3,6.8
3,Aberystwyth University,2012,2016,35.5,31.3,-4.2
4,Adam Mickiewicz University,2016,2016,28.7,28.7,0.0


Se *year_lr* = *year_mr* l'improvement è pari a 0 (banalmente)

Quando l'improvement è *NaN*, almeno uno dei due dati è mancante e quindi la differenza non è calcolabile per ovvi motivi.

### 3. Find the university with the largest increase computed in the previous point

In [15]:
least_most_improvement.sort_values(by='improvement',ascending=False).head(1)

,university_name_lr,year_lr,year_mr,income_lr,income_mr,improvement
470,TU Dresden,2012,2016,31.9,99.7,67.8


### 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for *Aarhus University* the value is 122-73=49). Notice that some rankings are expressed as a range

In [16]:
most_recent_s=shanghai.loc[shanghai.groupby('uni_id')['year'].idxmax()]
most_recent_c=cwur.loc[cwur.groupby('uni_id')['year'].idxmax()]

In [17]:
most_recent_ts=pd.merge(most_recent_t[['uni_id','world_rank']],most_recent_s[['uni_id','world_rank']], on='uni_id')
most_recent_all=pd.merge(most_recent_ts,most_recent_c[['uni_id','world_rank']], on='uni_id')
most_recent_all.head()

,uni_id,world_rank_x,world_rank_y,world_rank
0,aalborguniversity,201-250,301-400,565
1,aaltouniversity,251-300,401-500,421
2,aarhusuniversity,=106,73,122
3,aixmarseilleuniversity,251-300,101-150,206
4,aristotleuniversityofthessaloniki,601-800,401-500,459


Creazione di una funzione **norm__rank** per normalizzare le classifiche a range

In [18]:
def norm_rank(x):
    x=x.astype(str).str.strip('=').str.split('-',expand=True).astype(float).mean(axis=1)
    return x

In [19]:
most_recent_all['world_rank_t']=norm_rank(most_recent_all['world_rank_x'])
most_recent_all['world_rank_s']=norm_rank(most_recent_all['world_rank_y'])
most_recent_all['world_rank_c']=norm_rank(most_recent_all['world_rank'])
most_recent_all.drop(['world_rank_x','world_rank_y','world_rank'], axis=1, inplace=True)

In [20]:
most_recent_all.head()

,uni_id,world_rank_t,world_rank_s,world_rank_c
0,aalborguniversity,225.5,350.5,565.0
1,aaltouniversity,275.5,450.5,421.0
2,aarhusuniversity,106.0,73.0,122.0
3,aixmarseilleuniversity,275.5,125.5,206.0
4,aristotleuniversityofthessaloniki,700.5,450.5,459.0


In [21]:
max_rank=most_recent_all.max(axis=1)
min_rank=most_recent_all.min(axis=1)
most_recent_all['max_diff']=max_rank-min_rank
most_recent_all.head()

,uni_id,world_rank_t,world_rank_s,world_rank_c,max_diff
0,aalborguniversity,225.5,350.5,565.0,339.5
1,aaltouniversity,275.5,450.5,421.0,175.0
2,aarhusuniversity,106.0,73.0,122.0,49.0
3,aixmarseilleuniversity,275.5,125.5,206.0,150.0
4,aristotleuniversityofthessaloniki,700.5,450.5,459.0,250.0


### 5. Consider only the most recent data point of the *times* dataset. Compute the number of male and female students for each country.

In [22]:
most_recent_t.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,uni_id
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,NaN,17422.0,15.9,15%,48 : 52,2016,aalborguniversity
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,NaN,16099.0,24.2,17%,32 : 68,2016,aaltouniversity
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,aarhusuniversity
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,NaN,9252.0,19.2,18%,48 : 52,2016,aberystwythuniversity
2404,601-800,Adam Mickiewicz University,Poland,20.0,25.7,11.0,15.3,28.7,NaN,40633.0,15.6,1%,71 : 29,2016,adammickiewiczuniversity


In [23]:
fe_ma=most_recent_t['female_male_ratio'].str.split(':',expand=True).astype(float)
fe_ma.rename(columns={0:'female_perc',1:'male_perc'},inplace=True)
fe_ma

,female_perc,male_perc
2003,48.0,52.0
2056,32.0,68.0
1908,54.0,46.0
2105,48.0,52.0
2404,71.0,29.0
...,...,...
2601,28.0,72.0
2151,NaN,NaN
2152,59.0,41.0
2602,43.0,57.0


In [24]:
times_gender=pd.merge(most_recent_t,fe_ma,left_index=True,right_index=True)
times_gender.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,uni_id,female_perc,male_perc
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,NaN,17422.0,15.9,15%,48 : 52,2016,aalborguniversity,48.0,52.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,NaN,16099.0,24.2,17%,32 : 68,2016,aaltouniversity,32.0,68.0
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,aarhusuniversity,54.0,46.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,NaN,9252.0,19.2,18%,48 : 52,2016,aberystwythuniversity,48.0,52.0
2404,601-800,Adam Mickiewicz University,Poland,20.0,25.7,11.0,15.3,28.7,NaN,40633.0,15.6,1%,71 : 29,2016,adammickiewiczuniversity,71.0,29.0


In [25]:
times_gender['female_num']=round(times_gender['num_students']*times_gender['female_perc']/100)
times_gender['male_num']=times_gender['num_students']-times_gender['female_num']

In [26]:
times_gender.groupby('country')[['female_num','male_num']].sum()

,female_num,male_num
country,,
Argentina,67191.0,41182.0
Australia,391736.0,321640.0
Austria,68364.0,66113.0
Bangladesh,21323.0,41393.0
Belarus,20219.0,9084.0
...,...,...
Uganda,18670.0,18670.0
Ukraine,17846.0,19250.0
United Arab Emirates,9516.0,4931.0


### 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [27]:
times_gender['fm_ratio']=times_gender['female_num']/times_gender['male_num']
times_gender.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,uni_id,female_perc,male_perc,female_num,male_num,fm_ratio
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,NaN,17422.0,15.9,15%,48 : 52,2016,aalborguniversity,48.0,52.0,8363.0,9059.0,0.923170
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,NaN,16099.0,24.2,17%,32 : 68,2016,aaltouniversity,32.0,68.0,5152.0,10947.0,0.470631
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,aarhusuniversity,54.0,46.0,12903.0,10992.0,1.173854
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,NaN,9252.0,19.2,18%,48 : 52,2016,aberystwythuniversity,48.0,52.0,4441.0,4811.0,0.923093
2404,601-800,Adam Mickiewicz University,Poland,20.0,25.7,11.0,15.3,28.7,NaN,40633.0,15.6,1%,71 : 29,2016,adammickiewiczuniversity,71.0,29.0,28849.0,11784.0,2.448150


In [28]:
times_gender['fm_ratio'].mean() #inf

inf

Questo perchè in un'università il numero dei maschi è pari a 0, provocando appunto una media uguale a *Inf*

In [29]:
times_gender.sort_values(by='fm_ratio',ascending=False).head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,uni_id,female_perc,male_perc,female_num,male_num,fm_ratio
2227,401-500,Ewha Womans University,South Korea,23.9,34.3,18.3,50.4,57.6,NaN,17625.0,14.4,6%,100 : 0,2016,ewhawomansuniversity,100.0,0.0,17625.0,0.0,inf
2388,501-600,Soochow University,China,19.2,16.0,13.7,45.8,75.3,NaN,36465.0,12.9,2%,78 : 22,2016,soochowuniversity,78.0,22.0,28443.0,8022.0,3.545625
2413,601-800,Anadolu University,Turkey,12.2,14.3,22.6,10.9,100.0,NaN,379231.0,162.6,1%,78 : 22,2016,anadoluuniversity,78.0,22.0,295800.0,83431.0,3.545445
2036,201-250,Royal Veterinary College,United Kingdom,26.0,81.7,27.8,74.9,45.5,NaN,1819.0,10.9,18%,77 : 23,2016,royalveterinarycollege,77.0,23.0,1401.0,418.0,3.351675
2398,501-600,United Arab Emirates University,United Arab Emirates,18.8,95.0,14.8,21.7,30.1,NaN,9221.0,12.5,26%,76 : 24,2016,unitedarabemiratesuniversity,76.0,24.0,7008.0,2213.0,3.166742


Specificatamente l'università causa del problema è la Ewha Womans University. Come soluzione basta sostituire il valore *Inf* con un *NaN*

In [30]:
times_gender['fm_ratio'].replace(np.inf,np.nan,inplace=True)

In [31]:
avg=times_gender['fm_ratio'].mean()

In [32]:
below=times_gender[times_gender['fm_ratio']<avg]['university_name'].unique()

In [33]:
below

array(['Aalborg University', 'Aalto University', 'Aberystwyth University',
       'Ajou University', 'Alexandria University',
       'Aligarh Muslim University', 'American University of Beirut',
       'American University of Sharjah',
       'Amirkabir University of Technology', 'Andhra University',
       'Arizona State University',
       'Athens University of Economics and Business', 'Auburn University',
       'Austral University of Chile',
       'Bauman Moscow State Technical University', 'Bayreuth University',
       'Ben-Gurion University of the Negev',
       'Binghamton University, State University of New York',
       'Birla Institute of Technology and Science, Pilani',
       'Boğaziçi University', 'Brno University of Technology',
       'Brunel University London',
       'Budapest University of Technology and Economics',
       'Cairo University', 'California Institute of Technology',
       'Carleton University', 'Carlos III University of Madrid',
       'Carnegie Mellon

In [34]:
len(below)

346

### 7. For each country compute the fraction of students that are in one of the universities computed in the previous point.

In [35]:
total_students_country=times_gender.groupby('country')['num_students'].sum().to_frame()
total_students_country.head()

,num_students
country,
Argentina,108373.0
Australia,743627.0
Austria,134477.0
Bangladesh,62716.0
Belarus,29303.0


In [36]:
below_students_country=times_gender[times_gender['university_name'].isin(below)].groupby('country')['num_students'].sum().to_frame()
below_students_country.head()

,num_students
country,
Australia,146235.0
Austria,61033.0
Bangladesh,62716.0
Brazil,278774.0
Canada,86779.0


Si opera una left join poichè non è detto che tutti i paesi abbiano università il cui ratio sia sotto la media. In tal caso si sostituisce il valore *NaN* con 0

In [37]:
num_country=pd.merge(total_students_country,below_students_country, on='country',suffixes=['_total','_below'],how='left')
num_country.head()

,num_students_total,num_students_below
country,,
Argentina,108373.0,NaN
Australia,743627.0,146235.0
Austria,134477.0,61033.0
Bangladesh,62716.0,62716.0
Belarus,29303.0,NaN


In [38]:
num_country['num_students_below'].replace(np.nan,0,inplace=True)

In [39]:
num_country['below_perc']=num_country['num_students_below']/num_country['num_students_total']
num_country.head()

,num_students_total,num_students_below,below_perc
country,,,
Argentina,108373.0,0.0,0.000000
Australia,743627.0,146235.0,0.196651
Austria,134477.0,61033.0,0.453855
Bangladesh,62716.0,62716.0,1.000000
Belarus,29303.0,0.0,0.000000


### 8. Read the file `educational_attainment_supplementary_data.csv`, discarding any row without `country_name` or `series_name`

In [40]:
attainment=pd.read_csv('educational_attainment_supplementary_data.csv')
attainment.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


In [41]:
len(attainment)

79055

In [42]:
attainment.dropna(subset=['country_name','series_name'],inplace=True)
attainment.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


In [43]:
len(attainment)

79050

Complessivamente 5 righe scartate

### 9. From `attainment` build a dataframe with the same data, but with 4 columns: `country_name`, `series_name`, `year`, `value`

La funzione **melt** di pandas consente di trasformare tabelle da formato wide a long:
- id_vars: chiave primaria (identificativo)
- var_name: colonna variabile
- value_nam: colonna valore

In [44]:
new_attainment=pd.melt(attainment,id_vars=['country_name','series_name'],var_name='year',value_name='value')
new_attainment.head()

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54


### 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [45]:
list_uni=times['uni_id'].unique().tolist()+shanghai['uni_id'].unique().tolist()+cwur['uni_id'].unique().tolist()

La funzione **Counter** importata da collections prende in input una lista e crea un dizionario che assegna a ciascun elemento distinto (chiave) il numero di volte che questo compare nella lista (valore)

In [46]:
counter=coll.Counter(list_uni)
counter

Counter({'harvarduniversity': 3,
         'californiainstituteoftechnology': 3,
         'massachusettsinstituteoftechnology': 2,
         'stanforduniversity': 3,
         'princetonuniversity': 3,
         'universityofcambridge': 3,
         'universityofoxford': 3,
         'universityofcaliforniaberkeley': 3,
         'imperialcollegelondon': 2,
         'yaleuniversity': 3,
         'universityofcalifornialosangeles': 3,
         'universityofchicago': 3,
         'johnshopkinsuniversity': 3,
         'cornelluniversity': 3,
         'ethzurichswissfederalinstituteoftechnologyzurich': 1,
         'universityofmichigan': 1,
         'universityoftoronto': 3,
         'columbiauniversity': 3,
         'universityofpennsylvania': 3,
         'carnegiemellonuniversity': 3,
         'universityofhongkong': 2,
         'universitycollegelondon': 3,
         'universityofwashington': 2,
         'dukeuniversity': 3,
         'northwesternuniversity': 3,
         'universityoftokyo': 2,


In [47]:
len(counter)

1402

### 11. In the times ranking, compute the number of times each university appears

In [48]:
uni_times_counter=coll.Counter(times['uni_id'].tolist())

In [49]:
uni_times_counter

Counter({'harvarduniversity': 6,
         'californiainstituteoftechnology': 6,
         'massachusettsinstituteoftechnology': 6,
         'stanforduniversity': 6,
         'princetonuniversity': 6,
         'universityofcambridge': 6,
         'universityofoxford': 6,
         'universityofcaliforniaberkeley': 6,
         'imperialcollegelondon': 6,
         'yaleuniversity': 6,
         'universityofcalifornialosangeles': 6,
         'universityofchicago': 6,
         'johnshopkinsuniversity': 6,
         'cornelluniversity': 6,
         'ethzurichswissfederalinstituteoftechnologyzurich': 6,
         'universityofmichigan': 6,
         'universityoftoronto': 6,
         'columbiauniversity': 6,
         'universityofpennsylvania': 6,
         'carnegiemellonuniversity': 6,
         'universityofhongkong': 6,
         'universitycollegelondon': 6,
         'universityofwashington': 6,
         'dukeuniversity': 6,
         'northwesternuniversity': 6,
         'universityoftokyo': 6,


In [50]:
len(uni_times_counter)

818

### 12. Find the universities that appear at most twice in the times ranking.

In [51]:
twice={}
for key in uni_times_counter:
    if uni_times_counter[key]<=2:
        twice[key]=uni_times_counter[key]
twice

{'universityofwisconsin': 1,
 'medicaluniversityofsouthcarolina': 2,
 'universityofmedicineanddentistryofnewjersey': 1,
 'universityofhamburg': 1,
 'universityofkentucky': 2,
 'barilanuniversity': 2,
 'universityofhull': 2,
 'auburnuniversity': 2,
 'clemsonuniversity': 2,
 'georgiastateuniversity': 2,
 'griffithuniversity': 2,
 'harbininstituteoftechnology': 2,
 'hiroshimauniversity': 2,
 'kobeuniversity': 2,
 'kyungheeuniversity': 2,
 'latrobeuniversity': 2,
 'mahidoluniversity': 2,
 'michigantechnologicaluniversity': 1,
 'nationaltaiwanoceanuniversity': 2,
 'newjerseyinstituteoftechnology': 2,
 'newuniversityoflisbon': 2,
 'olddominionuniversity': 2,
 'polytechnicuniversityofcatalonia': 2,
 'polytechnicuniversityofturin': 2,
 'pontificalcatholicuniversityofchile': 2,
 'saintpetersburgstateuniversity': 2,
 'swanseauniversity': 2,
 'tokyouniversityofagricultureandtechnology': 2,
 'universityofcoimbra': 2,
 'universityofidaho': 2,
 'universityofkent': 2,
 'universityofparisnorthparis13'

In [52]:
len(twice)

408

### 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

Si richiama la funzione **norm_rank** costruita nel punto 4. Si utilizza la funzione copy() per evitare il warning *SettingWithCopyWarning*.

In [63]:
t_pos=times[['world_rank','university_name','year','uni_id']].copy()
t_pos['w_rank']=norm_rank(t_pos['world_rank'])
t_pos.head()

,world_rank,university_name,year,uni_id,w_rank
0,1,Harvard University,2011,harvarduniversity,1.0
1,2,California Institute of Technology,2011,californiainstituteoftechnology,2.0
2,3,Massachusetts Institute of Technology,2011,massachusettsinstituteoftechnology,3.0
3,4,Stanford University,2011,stanforduniversity,4.0
4,5,Princeton University,2011,princetonuniversity,5.0


In [64]:
s_pos=shanghai[['world_rank','university_name','year','uni_id']].copy()
s_pos['w_rank']=norm_rank(s_pos['world_rank'])
s_pos.head()

,world_rank,university_name,year,uni_id,w_rank
0,1,Harvard University,2005,harvarduniversity,1.0
1,2,University of Cambridge,2005,universityofcambridge,2.0
2,3,Stanford University,2005,stanforduniversity,3.0
3,4,"University of California, Berkeley",2005,universityofcaliforniaberkeley,4.0
4,5,Massachusetts Institute of Technology (MIT),2005,massachusettsinstituteoftechnologymit,5.0


In [65]:
c_pos=cwur[['world_rank','university_name','year','uni_id']].copy()
c_pos['w_rank']=norm_rank(c_pos['world_rank'])
c_pos.head()

,world_rank,university_name,year,uni_id,w_rank
0,1,Harvard University,2012,harvarduniversity,1.0
1,2,Massachusetts Institute of Technology,2012,massachusettsinstituteoftechnology,2.0
2,3,Stanford University,2012,stanforduniversity,3.0
3,4,University of Cambridge,2012,universityofcambridge,4.0
4,5,California Institute of Technology,2012,californiainstituteoftechnology,5.0


In [66]:
same_ts=pd.merge(t_pos,s_pos,on=['uni_id','year','w_rank'],suffixes=['_t','_s'])
same_ts

,world_rank_t,university_name_t,year,uni_id,w_rank,world_rank_s,university_name_s
0,1,Harvard University,2011,harvarduniversity,1.0,1,Harvard University
1,2,Stanford University,2012,stanforduniversity,2.0,2,Stanford University
2,9,University of Chicago,2012,universityofchicago,9.0,9,University of Chicago
3,11,Yale University,2012,yaleuniversity,11.0,11,Yale University
4,96,Michigan State University,2012,michiganstateuniversity,96.0,96,Michigan State University
5,2,Stanford University,2013,stanforduniversity,2.0,2,Stanford University
6,11,Yale University,2013,yaleuniversity,11.0,11,Yale University
7,15,University of Pennsylvania,2013,universityofpennsylvania,15.0,15,University of Pennsylvania
8,35,"University of California, Santa Barbara",2013,universityofcaliforniasantabarbara,35.0,35,"University of California, Santa Barbara"
9,6,Princeton University,2014,princetonuniversity,6.0,6,Princeton University


In [67]:
same_tsc=pd.merge(same_ts,c_pos,on=['uni_id','year','w_rank'])
same_tsc[['w_rank','year','uni_id','university_name']]

,w_rank,year,uni_id,university_name
0,2.0,2013,stanforduniversity,Stanford University
